Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [112]:
%run -i "assignment_helper.py"
%matplotlib inline

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


**Exercise 1** (1 point). We want to solve the equation $Ax=b$ with $A = \begin{bmatrix}1 & 0 \\0 & -1 \\ \end{bmatrix}$ and $b =\begin{bmatrix} 0 \\ 1 \\ \end{bmatrix}$ with quantum matrix inversion. We will encode $A$ in the unitary matrix $U=e^{iAt_0}$ with $t_0=\pi/2$, and $b$ in a register. With the ancilla (qubit 0), the eigenvalue registers (or the ancilla qubits of phase estimation, qubits 1 and 2), and the eigenstate, you will need a total of four qubits and one classical register for post-selection. Prepare the superposition in the eigenvalue register and the vector $b$. Place your solution in an object called `hhl`.

In [113]:
###
### YOUR CODE HERE
###
import numpy as np
import scipy.linalg
from grove.alpha.phaseestimation.phase_estimation import controlled
from pyquil import Program, get_qc
from pyquil.gates import *
#from forest_tools import *
qvm_server, quilc_server, fc = init_qvm_and_quilc('/home/local/bin/qvm', '/home/local/bin/quilc')
qc = get_qc('6q-qvm', connection=fc)
π = np.pi

#qc = get_qc('6q-qvm')
A = np.array([[1, 0], [0, -1]])
hhl = Program()

hhl += X(3)
hhl += I(0)
hhl += H(1)
hhl += H(2)

get_amplitudes(hhl)

array([0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j,
       0.5+0.j, 0. +0.j])

In [114]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j]))

**Exercise 2** (2 points). Start the quantum phase estimation by applying $C-U^{2^0}$ and $C-U^{2^1}$. Extend the circuit with the two appropriate gates.

In [115]:
###
### YOUR CODE HERE
###
import scipy.linalg
from grove.alpha.phaseestimation.phase_estimation import controlled

π = np.pi
hhl.defgate('CONTROLLED-U0', controlled(scipy.linalg.expm(1j*π*A)))
hhl += ('CONTROLLED-U0', 2, 3)
#hhl += CZ(2, 3)
hhl.defgate('CONTROLLED-U1', controlled(scipy.linalg.expm(2j*π*A)))
hhl += ('CONTROLLED-U1', 1, 3)
get_amplitudes(hhl)

array([ 0. +0.0000000e+00j,  0. +0.0000000e+00j,  0. +0.0000000e+00j,
        0. +0.0000000e+00j,  0. +0.0000000e+00j,  0. +0.0000000e+00j,
        0. +0.0000000e+00j,  0. +0.0000000e+00j,  0.5+0.0000000e+00j,
        0. +0.0000000e+00j,  0.5+1.2246468e-16j,  0. +0.0000000e+00j,
       -0.5-6.1232340e-17j,  0. +0.0000000e+00j, -0.5-1.8369702e-16j,
        0. +0.0000000e+00j])

In [116]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
                                          0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,  0.5+0.j,  0. +0.j,
                                         -0.5+0.j,  0. +0.j, -0.5+0.j,  0. +0.j]))

**Exercise 3** (1 point). Apply the quantum inverse Fourier transformation. Don't forget the swap just before the transformation.

In [117]:
###
### YOUR CODE HERE
###
hhl += SWAP(1, 2)
hhl += H(2)
hhl.defgate('CSdag', controlled(np.array([[1, 0], [0, -1j]])))
hhl += ('CSdag', 1, 2)
hhl += H(1)
get_amplitudes(hhl)

array([ 0.000000e+00+0.0000000e+00j,  0.000000e+00+0.0000000e+00j,
        0.000000e+00+0.0000000e+00j,  0.000000e+00+0.0000000e+00j,
        0.000000e+00+0.0000000e+00j,  0.000000e+00+0.0000000e+00j,
        0.000000e+00+0.0000000e+00j,  0.000000e+00+0.0000000e+00j,
        0.000000e+00-6.1232340e-17j,  0.000000e+00+0.0000000e+00j,
        1.000000e+00+1.8369702e-16j,  0.000000e+00+0.0000000e+00j,
        6.123234e-17-6.1232340e-17j,  0.000000e+00+0.0000000e+00j,
       -6.123234e-17-6.1232340e-17j,  0.000000e+00+0.0000000e+00j])

In [118]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
                                         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]))

**Exercise 4** (1 point). After this step, swap the eigenvalue registers again (this time to perform the inversion) and apply the controlled rotation. Use the same angle as in the lecture notebook.

In [119]:
###
### YOUR CODE HERE
###
hhl += SWAP(1, 2)
def rY(angle):
    '''Generate a rotation matrix over the Y axis in the Bloch sphere.
    
    :param angle: (float) The angle of rotation.

    :return: (numpy.ndarray) The rotation matrix
    '''
    return np.array([[np.cos(angle/2), -np.sin(angle/2)],
                     [np.sin(angle/2), np.cos(angle/2)]])

hhl.defgate('CRy0', controlled(rY(2*π/2**4)))
hhl += ('CRy0', 1, 0)
hhl.defgate('CRy1', controlled(rY(π/2**4)))
hhl += ('CRy1', 2, 0)

print(get_amplitudes(hhl))

[ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00-6.12323400e-17j  0.00000000e+00+0.00000000e+00j
  6.00557777e-17-6.00557777e-17j  1.19458369e-17-1.19458369e-17j
  9.95184727e-01+1.82812469e-16j  9.80171403e-02+1.80054566e-17j
 -5.85956960e-17-5.85956960e-17j -1.77748100e-17-1.77748100e-17j]


In [120]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j]))

**Exercise 5** (2 points). Uncompute the eigenvalue register

In [121]:
###
### YOUR CODE HERE
###
'''hhl += SWAP(1,2)
hhl += H(1)
hhl.defgate('CS', controlled(np.array([[1, 0], [0, 1j]])))
hhl += ('CS', 1, 2)
hhl += H(2)
hhl += SWAP(1,2)
hhl.defgate('CONTROLLED-U1dag', controlled(scipy.linalg.expm(-2j*π*A)))
hhl += ('CONTROLLED-U1dag', 1, 3)
hhl.defgate('CONTROLLED-U0dag', controlled(scipy.linalg.expm(-1j*π*A)))
hhl += ('CONTROLLED-U0dag', 2, 3)
hhl += H(2)
hhl += H(1)'''

hhl_2 = Program()
hhl_2 += CZ(2, 3)
hhl_2 += SWAP(1, 2)
hhl_2 += H(2)
hhl_2 += CPHASE(-np.pi/2, 1, 2)
hhl_2 += H(1)
hhl_2 += SWAP(1, 2)

hhl_2 = hhl_2.dagger()

hhl += hhl_2
hhl += hhl_2
get_amplitudes(hhl)

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        9.95184727e-01+1.82812469e-16j,  9.80171403e-02+1.80054566e-17j,
        0.00000000e+00-1.18651474e-16j,  0.00000000e+00-2.97206470e-17j,
       -3.06161700e-17+3.06161700e-17j,  0.00000000e+00+0.00000000e+00j,
        3.06161700e-17+3.06161700e-17j,  0.00000000e+00+0.00000000e+00j])

In [122]:
amplitudes

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00-6.12323400e-17j,  0.00000000e+00+0.00000000e+00j,
        6.00557777e-17-6.00557777e-17j,  1.19458369e-17-1.19458369e-17j,
        9.95184727e-01+1.82812469e-16j,  9.80171403e-02+1.80054566e-17j,
       -5.85956960e-17-5.85956960e-17j, -1.77748100e-17-1.77748100e-17j])

In [123]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]))

At this point, if we measure 1 in the ancilla (qubit 0), the state will be proportional to $A^{-1}b = \sum_j \beta_j\lambda_j^{-1}\left|u_j\right\rangle=0 \frac{1}{1} |0\rangle + 1 \frac{1}{-1} |1\rangle=-|1\rangle$.

In [124]:
hhl.measure(q, c)

backend = BasicAer.get_backend('qasm_simulator') # the device to run on
result = execute(hhl, backend, shots=100).result()
counts  = result.get_counts(hhl)
print(counts)

NameError: name 'q' is not defined